# 训练量子隐形传态协议

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概述

量子隐形传态（quantum teleportation）是可以通过本地操作和经典通信（LOCC）协议完成的另一项重要任务，该协议借助提前制备好的纠缠资源在两个空间上分离的通信节点（仅允许经典信道）之间传输量子信息。在本教程中，我们将首先简要回顾一下量子隐形传态协议，并使用量桨进行模拟。然后，我们将介绍如何使用 LOCCNet 学习出一个量子隐形传态协议。

## 量子隐形传态协议

量子隐形传态协议最初由 C. H. Bennett 等人在 1993 年提出 [1]，并在 1997 年通过基于光子的实验平台进行了验证 [2-3]。其主要工作流程如下图所示。传输过程中需要 2 个通信节点或参与方，即 Alice 和 Bob。简单起见，我们仅考虑传输一个单量子比特的量子态 $|\psi\rangle_C$，这样整个系统总共需要 3 个量子比特，包括提前共享给 Alice 和 Bob 的最大纠缠态 $|\Phi^+\rangle_{AB} = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$。 Alice 持有量子比特 A 和 C， Bob 持有量子比特 B。**注释：量子隐形传态协议仅传输量子信息，而非在物理上直接传输量子比特。**

![](figures/teleportation-fig-circuit.jpg "图 1：量子隐形传态: 传输量子态 $|\psi\rangle$ 从 Alice 到 Bob")
<div style="text-align:center">图 1：量子隐形传态: 传输量子态 $|\psi\rangle$ 从 Alice 到 Bob </div>

**步骤 I：** 一开始，整个量子系统的状态可以描述为

$$
\lvert\varphi_{0}\rangle 
= \lvert\psi\rangle_{C}\otimes \lvert\Phi^+\rangle_{AB} 
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert00\rangle + \lvert11\rangle)\big],
\tag{1}
$$

 其中 Alice 要传输的量子态是 $|\psi\rangle_C = \alpha|0\rangle_C + \beta|1\rangle_C$ 并且 $\alpha, \beta \in \mathbb{C}$。

**步骤 II：**  Alice 在她持有的两个量子比特上作用 CNOT 门，得到作用后的量子态为

$$
|\varphi_1\rangle  
= \frac{1}{\sqrt{2}}\big[\alpha\lvert0\rangle(\lvert00\rangle + \lvert11\rangle)+\beta\lvert1\rangle(\lvert10\rangle + \lvert01\rangle)\big],
\tag{2}
$$

**步骤 III：** 接着 Alice 在她持有的量子比特 $C$ 上作用 Hadamard 门，使得整个系统的状态变为 $|\varphi_2\rangle$

$$
|\varphi_2\rangle = \frac{1}{2}\big[\alpha(\lvert0\rangle + \lvert1\rangle)(\lvert00\rangle + \lvert11\rangle)+\beta(\lvert0\rangle - \lvert1\rangle)(\lvert10\rangle + \lvert01\rangle)\big],
\tag{3}
$$

为了更好地观察后续的测量结果，不妨将上述态重新写为

$$
\lvert\varphi_{2}\rangle = \frac{1}{2}\big[\lvert00\rangle(\alpha\lvert0\rangle + \beta\lvert1\rangle) + \lvert01\rangle(\alpha\lvert1\rangle + \beta\lvert0\rangle) + \lvert10\rangle(\alpha\lvert0\rangle - \beta\lvert1\rangle) + \lvert11\rangle(\alpha\lvert1\rangle - \beta\lvert0\rangle)\big].
\tag{4}
$$

**步骤 IV：**  Alice 在计算基（computational basis）$\{|00\rangle, |01\rangle, |10\rangle, |11\rangle\}$ 上测量她的两个量子比特并将结果 $m_1m_2$ 通过经典信道发送给 Bob 。 总共有 4 种不同的可能性： $m_1m_2 \in \{ 00, 01,10, 11\}$。 然后， Bob 根据收到的消息在其量子比特 $B$ 上进行对应的操作。

- 如果测量结果为 $m_1m_2 = 00$，则 Bob 的态将为 $\alpha\lvert0\rangle + \beta\lvert1\rangle$。 无需任何操作即可完成传输。
- 如果测量结果为 $ m_1m_2 = 01 $，则 Bob 的态将为 $\alpha\lvert1\rangle + \beta\lvert0\rangle$。  Bob 需要在其量子比特上作用 $ X $ 门。
- 如果测量结果为 $ m_1m_2 = 10 $，则 Bob 的态将为 $\alpha\lvert0\rangle - \beta\lvert1\rangle$。  Bob 需要在其量子比特上作用 $ Z $ 门。
- 如果测量结果为 $ m_1m_2 = 11 $，则 Bob 的态将为 $\alpha\lvert1\rangle - \beta\lvert0\rangle$。 Bob 需要在其量子比特上执行 $ X $ 门操作，然后执行 $ Z $ 门操作。

在下一节中，我们将介绍如何使用量桨模拟量子隐形传态协议。

## Paddle Quantum 代码实现

首先，我们需要导入所有依赖包：

In [2]:
import numpy as np
import paddle
from paddle import matmul, trace
import paddle_quantum
from paddle_quantum.locc import LoccNet
from paddle_quantum.qinfo import state_fidelity
from paddle_quantum.state import bell_state, isotropic_state, random_state
# 切换至密度矩阵模式
paddle_quantum.set_backend('density_matrix')

初始化整个量子系统，然后定义量子电路和隐形传态协议。

In [3]:
class LOCC(LoccNet):
    def __init__(self):
        super(LOCC, self).__init__()
        # 添加第一个参与方 Alice
        # 第一个参数 2 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(2, party_name="Alice")
        # 添加第二个参与方 Bob
        # 第一个参数 1 代表着 Bob 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name="Bob")

        # 准备一个贝尔态
        _state = bell_state(2)
        # _state = isotropic_state(2, 0.8)     
        # 随机制备传输用的纯态（rank=1）
        self.state_C = random_state(num_qubits=1, rank=1)

        # 通过分配上述制备好的量子态初始化整个量子系统
        # 这里 ("Alice", 0) 即表示量子比特 C
        # 这里 ("Alice", 1) 即表示量子比特 A
        # 这里 ("Bob", 0) 即表示量子比特 B
        # print('提前分配好的纠缠态为:\n', _state.numpy())
        self.set_init_state(self.state_C, [("Alice", 0)])
        self.set_init_state(_state, [("Alice", 1), ("Bob", 0)])

        # 设置 Alice 的本地操作
        self.cirA = self.create_ansatz("Alice")
        self.cirA.cnot([0, 1])
        self.cirA.h(0)
        # 创建 Bob 的本地操作
        self.cirB = [self.create_ansatz("Bob") for _ in range(4)]
        self.cirB[1].x(0)
        self.cirB[2].z(0)
        self.cirB[3].x(0)
        self.cirB[3].z(0)

    def teleportation(self):
        status = self.init_status
        # 运行上述电路
        status = self.cirA(status)
        # Alice 在计算基上测量她所持有的两个量子比特 C 还有 A
        # 得到并记录四种结果 00，01，10，11
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])

        # 用于记录平均保真度
        fid_list = []
        # Bob 根据 Alice 的测量结果选择不同的门作用在自己的量子比特上
        for i, s in enumerate(status_A):
            # 根据 Alice 的测量结果，进行不同操作
            cirB = self.cirB[int(status_A[i].measured_result, 2)]
            # 执行电路
            status_B = cirB(s)
            # 仅保留 Bob 的量子比特 B
            status_fin = self.partial_state(status_B, [("Bob", 0)])
            # 计算初始态和传输后态之间的保真度
            fid = state_fidelity(self.state_C, status_fin) ** 2
            fid_list.append(fid * status_fin.prob)
        fid_avg = sum(fid_list)

        return fid_avg

然后，我们随机生成 200 个量子纯态，并使用态保真度 $ F $ 来衡量传输协议好坏，其中

$$
F(\rho,\sigma) \equiv \text{tr}\big( \sqrt{\sqrt{\rho}\sigma \sqrt{\rho}} \big)^2.
\tag{5}
$$

In [5]:
SEED = 999              # 固定随机数
num_state = 200         # 设置随机态的生成数量
list_fid = []           # 用于记录保真度
np.random.seed(SEED)
# 开始采样 
for idx in range(num_state):
    list_fid.append(LOCC().teleportation().numpy())

print('平均保真度 =', np.around(sum(list_fid)[0] / len(list_fid), 4), ', 标准差 =', np.std(list_fid))

平均保真度 = 1.0 , 标准差 = 4.0480725e-07


**注释：** 我们要指出的是，该协议的有效性取决于提前分配好的贝尔态的品质。 感兴趣的读者可以将纠缠态从 `bell_state(2)` 更改为 `isotropic_state(2, p)`，并测试观察在贝尔态中出现的量子噪声将如何影响该协议的性能。 其中 isotropic 态的定义如下，

$$
\rho_{\text{iso}}(p) = p\lvert\Phi^+\rangle \langle\Phi^+\rvert + (1-p)\frac{I}{4}, \quad p \in [0,1]
\tag{6}
$$

## 使用 LOCCNet 学习量子隐形传态

### 训练一个自定义的 LOCC 协议

一般的 LOCC 协议可以通过经典通信的回合数 $ r $ 来进行分类。 原始的量子隐形传态协议是单轮通讯协议（$ r = 1 $）。 为简单起见，我们也将通信回合数限制为单轮。与原始的协议不同，我们将使用参数化量子电路（PQC）来把 Bob 作用在量子比特上的固定门 $U\in\{X,Z\}$ 替换成布洛赫球面上的一个广义旋转门 $ U_3 $， 其定义为

$$
U_3(\theta, \phi, \varphi) =
\begin{bmatrix}
\cos(\frac{\theta}{2})           & -e^{i\varphi}\sin(\frac{\theta}{2})\\
e^{i\phi}\sin(\frac{\theta}{2})  & e^{i(\phi+\varphi)} \cos(\frac{\theta}{2})
\end{bmatrix}.
\tag{7}
$$

这将为我们带来更强大的搜索能力，来寻找更多使用场景下的 LOCC 协议。类似地，我们将 Alice 的本地操作更改为更通用的参数化量子电路，这里具体使用的是一个内置的两量子比特通用门 `universal_two_qubits()` [4]。基于以上条件，我们训练一个自定义的 LOCC 协议流程如下：

1. Alice 对她所持有的两个量子比特作用两量子比特通用门。
2. 然后 Alice 在计算基上测量她的两个量子比特，并通过经典信道与 Bob 交流。
3. 共计有 4 种可能的测量结果：$m_1m_2 \in \{00, 01, 10, 11\}$。 Bob 需要根据这些测量结果采取不同的本地操作。在 Bob 进行操作后，记其量子态为 $\lvert \psi\rangle_{B}$。
4. 计算 $\lvert \psi\rangle_{B}$ 与 $\lvert\psi\rangle_C$（纯态）之间的量子态重叠（state overlap）并记为 $ O $。 由于 LOCCNet 框架目前仅支持密度矩阵形式，因此我们必须将它们重写为 $\rho_{B} = |\psi\rangle\langle\psi|_B$ 和 $\rho_{C} = |\psi\rangle\langle\psi|_C$。然后可以得到 $O = \text{Tr}(\rho_C\rho_{B})$。对于纯态，此距离度量就是保真度 (fidelity)。
5. 将损失函数设置为 4 种可能测量结果的累加，即 $L = \sum_{m_1m_2} \big(1-\text{Tr}(\rho_C\rho_{B})\big)$，并使用基于梯度的优化方法更新 Alice 和 Bob 本地操作中的参数，从而使得损失函数最小化。
6. 重复步骤 1-5，直到损失函数收敛。
7. 生成一组随机的态 $\{\lvert\psi_C\rangle\}$，并以平均保真度对训练出的传输协议进行基准测试。

![](figures/teleportation-fig-LOCCNet.png "图 2： 用 LOCCNet 学习量子隐形传态协议")
<div style="text-align:center">图 2： 用 LOCCNet 学习量子隐形传态协议 </div>

**注释：** 为了确保训练出的 LOCC 协议对所有态均有效，我们将训练集设为 4 个线性独立态，即 $\{|0\rangle\langle 0|,|1\rangle\langle 1|,|+\rangle\langle +|,|+\rangle\langle +|_y\}$ 。其中 $|+\rangle, |+\rangle_y$ 分别为泡利 $X,Y$ 矩阵的正特征值对应的特征向量。 该训练集在密度矩阵的表示下为：

$$
\rho_0 = \left[\begin{array}{ccc}
1 & 0\\
0 & 0
\end{array}\right], 
\rho_1 = \left[\begin{array}{ccc}
0 & 0\\
0 & 1
\end{array}\right], 
\rho_2 = \left[\begin{array}{ccc}
0.5 & 0.5\\
0.5 & 0.5
\end{array}\right], 
\rho_3 = \left[\begin{array}{ccc}
0.5 & -0.5 i\\
0.5i & 0.5
\end{array}\right]. 
\tag{8}
$$

任何一个单量子比特量子态都可以写为上述 4 个态的线性组合。

In [6]:
class LOCC_Train(LoccNet):
    def __init__(self):
        super(LOCC_Train, self).__init__()
        # 初始化 LOCCNet
        self.parties = list()
        # 添加第一个参与方 Alice
        # 第一个参数 2 代表着 Alice 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(2, party_name="Alice")
        # 添加第二个参与方 Bob
        # 第一个参数 1 代表着 Bob 手里有几个量子比特
        # 第二个参数代表着参与方的名字
        self.add_new_party(1, party_name="Bob")

        # 准备一个贝尔态
        _state = bell_state(2)
        # _state = isotropic_state(2, 0.8)
        # 训练集: 4 个线性独立态
        _state0 = paddle_quantum.State(np.array([[1, 0], [0, 0]], dtype=np.complex64))
        _state1 = paddle_quantum.State(np.array([[0, 0], [0, 1]], dtype=np.complex64))
        _state2 = paddle_quantum.State(np.array([[0.5, 0.5], [0.5, 0.5]], dtype=np.complex64))
        _state3 = paddle_quantum.State(np.array([[0.5, -0.5j], [0.5j, 0.5]], dtype=np.complex64))
        self.init_states = [_state0, _state1, _state2, _state3]
        # 通过分配上述制备好的量子态初始化整个量子系统
        self.set_init_state(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_state(_state0, [("Alice", 0)])

        # 定义 Alice 的本地操作
        self.cirA = self.create_ansatz("Alice")
        self.cirA.universal_two_qubits([0, 1])
        # 定义 Bob 的本地操作
        self.cirB = [self.create_ansatz("Bob") for _ in range(4)]
        for cir in self.cirB:
            # 作用单量子比特通用门
            cir.u3(0)

    def LOCCNet(self):
        # 定义训练过程
        loss = 0
        temp_state = self.init_status
        # 开始训练
        for init_state in self.init_states:
            # 重置 Alice 持有的量子比特 C 至训练集中的量子态
            status = self.reset_state(temp_state, init_state, [("Alice", 0)])
            # 执行 Alice 的电路
            status = self.cirA(status)
             # 测量得到四个可能的结果
            status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
            
            # Bob 根据测量结果选择不同的门作用在自己的量子比特上
            for i, s in enumerate(status_A):
                # 执行 Bob 的电路
                status_B = self.cirB[i](s)
                # 仅留下 Bob 的量子比特 B
                status_fin = self.partial_state(status_B, [("Bob", 0)])
                # 将所有的测量结果的损失函数进行累加
                loss += 1 - paddle.real(trace(matmul(init_state.data, status_fin.data)))
        
        return loss

    # 存储训练结束后的最优参数
    def save_module(self):
        theta_A = self.cirA.parameters()
        theta_B = [self.cirB[i].parameters() for i in range(4)]
        theta = theta_A + sum(theta_B, [])
        paddle.save(theta, 'parameters/QT_LOCCNet')

In [9]:
ITR = 150   # 设置优化循环次数
LR = 0.2    # 设置学习速率
SEED = 999  # 固定本地操作中参数的初始化随机数种子
np.random.seed(SEED)
paddle.seed(SEED)

net = LOCC_Train()
params = net.cirA.parameters() + sum([net.cirB[i].parameters() for i in range(4)], [])
# 选择 Adam 优化器
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=params)
# 开始优化循环
for itr in range(ITR):
    # 向前传播计算损失函数
    loss = net.LOCCNet()
    # 反向传播优化损失函数
    loss.backward()
    opt.minimize(loss)
    # 清除梯度
    opt.clear_grad()
    if itr % 10 == 0:
        print("itr " + str(itr) + ":", loss.numpy()[0])

# 保存参数
net.save_module()

itr 0: 7.721435
itr 10: 0.5636051
itr 20: 0.20643002
itr 30: 0.073403895
itr 40: 0.025113285
itr 50: 0.008655369
itr 60: 0.0035846233
itr 70: 0.0014438629
itr 80: 0.00045502186
itr 90: 0.00018626451
itr 100: 4.7028065e-05
itr 110: 1.4960766e-05
itr 120: 4.4703484e-06
itr 130: 3.1590462e-06
itr 140: 1.66893e-06


### 基准测试

如果想要更快捷地感受效果，我们还提供**加载预先训练好的电路参数**并直接测试性能。

In [10]:
class LOCC_Test(LoccNet):
    def __init__(self, theta_A, theta_B):
        super(LOCC_Test, self).__init__()
        self.parties = list()
        self.add_new_party(2, party_name="Alice")
        self.add_new_party(1, party_name="Bob")

        _state = bell_state(2)
        self._state0 = random_state(1)
        self.set_init_state(_state, [("Alice", 1), ("Bob", 0)])
        self.set_init_state(self._state0, [("Alice", 0)])

        self.cirA = self.create_ansatz("Alice")
        self.cirA.universal_two_qubits(qubits_idx=[0, 1], param=theta_A)
        self.cirB = [self.create_ansatz("Bob") for _ in range(4)]
        for i, cir in enumerate(self.cirB):
            cir.u3(qubits_idx=0, param=theta_B[i])

    def benchmark(self):
        input_state = self.init_status
        status = self.cirA(input_state)
        status_A = self.measure(status, [("Alice", 0), ("Alice", 1)], ["00", "01", "10", "11"])
        fid_list = []
        for i, s in enumerate(status_A):
            status_B = self.cirB[i](s)
            status_fin = self.partial_state(status_B, [("Bob", 0)])
            fid = state_fidelity(self._state0, status_fin) ** 2
            fid_list.append(fid * status_fin.prob)
        fid_ave = sum(fid_list)
        
        return fid_ave

In [11]:
# 加载预先训练的电路参数
para = paddle.load('parameters/QT_LOCCNet')
SEED = 999              # 固定生成传输态的随机种子
num_state = 200         # 设置随机态的生成数量
list_fid = []           # 用于记录保真度
np.random.seed(SEED)
paddle.seed(SEED)

# 采样
for idx in range(num_state):
    list_fid.append(LOCC_Test(para[0], para[1:]).benchmark().numpy())

print('平均保真度 =', np.around(sum(list_fid)[0] / len(list_fid), 4), ', 标准差 =', np.std(list_fid))

平均保真度 = 1.0 , 标准差 = 3.8644316e-07


## 结论

从以上数值实验的结果可以看出，基于 LOCCNet 我们成功地学习出了量子隐形传态协议。 最初的隐形传态协议旨在传输单量子比特量子态，无法直接推广到多量子比特的情形。 相比之下，LOCCNet 为寻找多量子比特情形下的隐形传态协议提供了可能，更是可以训练得到更一般的通过量子纠缠和 LOCC 实现不同节点间量子信道的模拟。 此外，感兴趣的读者可以尝试各类量子噪声会如何影响 LOCCNet 训练的效果。另一方面，隐形传态协议可以看作是在模拟一个单位信道 $\mathcal{E}_I$，即 Alice 发送量子态 $\psi$ 而 Bob 接收到量子态 $\mathcal{E}_I(\psi)$。这个想法可以扩展到模拟许多其他信道，例如去极化信道 $\mathcal{E}_{D}$。

---

## 参考文献

[1] Bennett, Charles H., et al. "Teleporting an unknown quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 70.13 (1993): 1895.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895)

[2] Boschi, Danilo, et al. "Experimental realization of teleporting an unknown pure quantum state via dual classical and Einstein-Podolsky-Rosen channels." [Physical Review Letters 80.6 (1998): 1121.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.1121)

[3] Bouwmeester, Dik, et al. "Experimental quantum teleportation." [Nature 390.6660 (1997): 575-579.](https://www.nature.com/articles/37539)

[4] Vidal, Guifre, and Christopher M. Dawson. "Universal quantum circuit for two-qubit transformations with three controlled-NOT gates." [Physical Review A 69.1 (2004): 010301.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.69.010301)